In [1]:
import pandas as pd
import numpy as np
import re
import ast

In [9]:
cellphones = pd.read_csv(r'D:\Code\DS108\crawling\cellphones\raw_data.csv', encoding='utf-8')
tgdd = pd.read_csv(r'D:\Code\DS108\crawling\tgdd\raw_data.csv', encoding='utf-8')

In [27]:
data = pd.concat([cellphones, tgdd], ignore_index=True)

In [11]:
# Kiểm tra số cột và dòng của dữ liệu
print(data.shape)

(13760, 23)


In [12]:
# Kiểm tra tên các cột của dữ liệu
print(data.columns)

Index(['name', 'brand', 'color', 'condition', 'price_old', 'price_new',
       'image', 'warranty', 'CPU', 'RAM', 'capacity', 'time', 'battery',
       'screen_size', 'operating_system', 'display_technology',
       'screen_resolution', 'SIM', 'size', 'weight', 'bluetooth',
       'refresh_rate', 'GPU'],
      dtype='object')


# Chuyển đổi kiểu dữ liệu

In [13]:
# Kiểu dữ liệu của thuộc tính
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13760 entries, 0 to 13759
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                12628 non-null  object 
 1   brand               12328 non-null  object 
 2   color               12609 non-null  object 
 3   condition           12563 non-null  object 
 4   price_old           12031 non-null  object 
 5   price_new           12005 non-null  object 
 6   image               12628 non-null  object 
 7   warranty            11950 non-null  object 
 8   CPU                 12474 non-null  object 
 9   RAM                 12320 non-null  object 
 10  capacity            12603 non-null  object 
 11  time                11744 non-null  object 
 12  battery             12619 non-null  object 
 13  screen_size         12617 non-null  object 
 14  operating_system    10505 non-null  object 
 15  display_technology  11845 non-null  object 
 16  scre

In [29]:
# Chuyển đổi cột 'price_old' từ object sang numeric
data['price_old'] = data['price_old'].astype(str)
data['price_old'] = data['price_old'].str.replace('.', '', regex=False)
data['price_old'] = data['price_old'].str.replace('₫', '', regex=False)
data['price_old'] = data['price_old'].str.replace('đ', '', regex=False)
data['price_old'] = pd.to_numeric(data['price_old'], errors='coerce')

In [31]:
# Chuyển đổi cột 'price_new' từ object sang numeric
data['price_new'] = data['price_new'].astype(str)
data['price_new'] = data['price_new'].str.replace('.', '', regex=False)
data['price_new'] = data['price_new'].str.replace('₫', '', regex=False)
data['price_new'] = data['price_new'].str.replace('đ', '', regex=False)
data['price_new'] = pd.to_numeric(data['price_new'], errors='coerce')


# Xử lí dữ liệu trùng lắp

In [33]:
# Kiểm tra duplicate trong dữ liệu
data.duplicated().sum()

1292

Một số máy có thông số giống nhau hoàn toàn -> Drop.

In [34]:
# Drop các dòng bị duplicated trong dữ liệu
data.drop_duplicates(inplace=True)

# Xử lí các dòng rỗng hoàn toàn

In [35]:
# Tìm các dòng rỗng hoàn toàn (tất cả các cột đều là NaN)
empty_rows = data.isnull().all(axis=1)
print(empty_rows.sum())

1


In [36]:
# Drop các dòng rỗng hoàn toàn trong dữ liệu
data.drop(index=data[pd.Series(empty_rows).values].index, inplace=True)

# Xử lí cột name

In [37]:
print(data['name'].unique())

['iPhone 14 Pro Max 128GB' 'iPhone 13 Pro Max 128GB'
 'OPPO Find N3 Flip 12GB 256GB' 'Samsung Galaxy S24 Plus 12GB 256GB'
 'Samsung Galaxy S24 Ultra 12GB 256GB' 'iPhone 14 Pro Max 256GB'
 'iPhone 14 Pro 128GB' 'iPhone 13 Pro Max 256GB' 'iPhone 15 Plus 128GB'
 'Samsung Galaxy S23 Ultra 8GB 256GB' 'OPPO A78 8GB 256GB'
 'OPPO Find N3 16GB 512GB' 'iPhone 15 Pro 128GB' 'iPhone 15 Pro Max 256GB'
 'Samsung Galaxy S23 8GB 128GB'
 'iPhone 16 Plus 128GB Chính hãng VN/A (3N350)' 'iPhone 11 64GB'
 'iPhone 8 Plus 64GB cũ đẹp' 'iPhone 15 Pro 256GB'
 'Samsung Galaxy S22 Ultra (12GB' 'iPhone XS Max 256GB Cũ đẹp'
 'iPhone 12 Pro Max 128GB' 'Samsung Galaxy Z Fold5 12GB 512GB'
 'Apple iPhone 7 Plus 128GB' 'Samsung Galaxy M55 (12GB 256GB)'
 'iPhone 14 128GB' 'iPhone XS Max 64GB Cũ trầy xước' 'iPhone 11 128GB'
 'iPhone 16 Pro Max 256GB | Chính hãng VN/A' 'iPhone 15 128GB'
 'iPhone 14 Plus 128GB' 'iPhone 13 256GB'
 'Samsung Galaxy A35 5G 8GB 128GB' 'Xiaomi 13T Pro 5G (12GB'
 'Samsung Galaxy Z Flip5 512GB' '

Cần loại bỏ các thông tin không cần thiết

In [38]:
data['name'] = data['name'].str.replace(r'Điện thoại ', '', regex=True).str.strip()

data = data.drop(data[data['name'] == 'false'].index)

pattern_to_remove = r'\s*(?:' \
                    r'\d+\s*GB' \
                    r'|\d+\s*RAM' \
                    r'|\d+\s*ROM' \
                    r'|\d+GB:\d+GB' \
                    r'|\d+:\d+' \
                    r'|Cũ' \
                    r'|\d+\s*TB' \
                    r'|Đã' \
                    r'|\b\d{4}\b' \
                    r'|2\s*sim' \
                    r'|\(' \
                    r'|Chính' \
                    r').*$'

data['name'] = data['name'].str.replace(pattern_to_remove, '', regex=True, flags=re.IGNORECASE)
data['name'] = data['name'].str.strip()

normalization_map = {
    'samsung': 'Samsung',
    'honor': 'HONOR',
    'mobell': 'Mobell',
    'xiaomi': 'Xiaomi',
    'iphone': 'iPhone',
    'oppo': 'OPPO',
    'nokia': 'Nokia',
    'masstel': 'Masstel',
    'realme': 'realme',
    'vivo': 'vivo',
    'itel': 'Itel',
    'tecno': 'Tecno',
    'benco': 'BENCO',
    'oneplus': 'OnePlus',
    'find': 'Find',
    'spark': 'Spark',
    'poco': 'Poco',
    'camon': 'Camon',
    'pova': 'Pova'
}

updated_names = []
for index, row in data.iterrows():
    product_name = row['name']
    
    if pd.isna(product_name):
        updated_names.append(np.nan)
        continue

    product_name_lower = str(product_name).lower()
    
    found_and_replaced = False
    for lower, normalized in normalization_map.items():
        pattern = r'\b' + re.escape(lower) + r'\b'
        
        if re.search(pattern, product_name_lower):
            product_name = re.sub(pattern, normalized, product_name, flags=re.IGNORECASE)
            found_and_replaced = True 
    
    updated_names.append(product_name)
    
data['name'] = updated_names
data['name'] = data['name'].str.replace(r'\bApple\s*', '', regex=True, flags=re.IGNORECASE)
data['name'] = data['name'].str.strip()

# Xử lí missing values

In [39]:
data.isnull().sum()

name                     0
brand                  300
color                   18
condition               64
price_old              594
price_new              620
image                    0
warranty               668
CPU                    152
RAM                    303
capacity                25
time                   874
battery                  9
screen_size             11
operating_system      2074
display_technology     778
screen_resolution      203
SIM                    114
size                   154
weight                1797
bluetooth              343
refresh_rate          2376
GPU                   1972
dtype: int64

Thuộc tính price_old là thuộc tính rất quan trọng, được xem là target của bài toán -> Drop các dòng bị thiếu thuộc tính price_old

In [40]:
# Loại bỏ các dòng bị thiếu giá trị trong cột 'price_old'
data = data.dropna(subset=['price_old'])

Điền giá trị thiếu dựa trên các giá trị có sẵn 

In [55]:
for name_value, group in data.groupby('name'):
    total = len(group)
    columns_to_check = ["CPU", "RAM", "capacity", "time", "battery", "screen_size", "operating_system", 
                    "display_technology", "screen_resolution", "SIM", "size", "weight", "bluetooth", 
                    "refresh_rate", "GPU"]
    
    for column in columns_to_check:
        missing = group[column].isna().sum()
        ratio = missing / total

        if 0 < ratio < 1:
            # Lấy giá trị đầu tiên không bị thiếu trong nhóm
            value = group[column].dropna().iloc[0]
            # Gán lại cho các dòng bị thiếu trong nhóm đó
            data.loc[(data['name'] == name_value) & (data[column].isna()), column] = value

Điền thiếu bằng cách tra các thông tin thiếu trên PhoneArena

In [47]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time

def scrape_from_phonearena(driver, name, keywords):
    # Bấm vào icon tìm kiếm
    search_icon = driver.find_element(By.CSS_SELECTOR, ".ic.ic-search.ic-24")
    search_icon.click()

    # Nhập từ khóa vào ô tìm kiếm
    search_box = driver.find_element(By.XPATH, "//input[@class = 'input input-search']")
    search_box.send_keys(name)
    search_box.send_keys(Keys.RETURN)

    # Bấm vào link đầu tiên trong danh sách kết quả
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".title a")))
    for attempt in range(3):
        try:
            # Tìm phần tử lại trước mỗi lần thử
            first_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".title a")))
            driver.execute_script("arguments[0].scrollIntoView();", first_link)
            time.sleep(5)
            first_link.click()
            break  # Nếu click thành công, thoát vòng lặp
        except StaleElementReferenceException:
            print(f"Retrying click due to stale element (attempt {attempt + 1})...")
        except Exception as e:
            print(f"Unexpected error: {e}")  # Bắt lỗi khác nếu có
    else:
        raise Exception("Failed to click first search result after 3 attempts due to StaleElementReferenceException")

        
    result = [None] * len(keywords)
    info_labels = {
        "CPU": "Systems chip", 
        "RAM": "RAM", 
        "capacity": "Internal storage", 
        "time": "",
        "battery": "Capacity", 
        "screen_size": "Dimensions", 
        "operating_system": "OS", 
        "display_technology": "Technology", 
        "screen_resolution": "Resolution", 
        "SIM": "SIM type", 
        "size": "Size", 
        "weight": "Weight", 
        "bluetooth": "Bluetooth", 
        "refresh_rate": "Refresh rate", 
        "GPU": "GPU"
    }
    
    try:
        for idx, keyword in enumerate(keywords):
            label = info_labels[keyword]
            if label != 'time':
                xpath = f"//th[contains(text(), '{label}')]//following-sibling::td"
            else:
                xpath = f"//span[contains(text(), 'Released')]//following-sibling::span[2]"
            try:
                element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                result[idx] = element.text
            except: 
                pass
    except:
        return result
        
    return result

In [53]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get("https://www.phonearena.com/")
time.sleep(30)

# Cột cần kiểm tra
columns_to_check = ["CPU", "RAM", "capacity", "time", "battery", "screen_size", "operating_system", 
                    "display_technology", "screen_resolution", "SIM", "size", "weight", "bluetooth", 
                    "refresh_rate", "GPU"]

for phone in data['name'].unique():
    mask = (data['name'] == phone)  # Lọc tất cả các dòng có cùng `name`
    missing_columns = [col for col in columns_to_check if data.loc[mask, col].isnull().any()]  

    if missing_columns:
        try:
            print(f"Scraping {phone} cho các thuộc tính bị thiếu: {missing_columns}")
            specs = scrape_from_phonearena(driver, phone, missing_columns)

            # Điền dữ liệu vào tất cả các dòng có cùng `name`
            for i, col in enumerate(missing_columns):
                if specs[i] is not None:
                    data.loc[mask & data[col].isnull(), col] = specs[i]
        except:
            pass

driver.quit()



Scraping Samsung Galaxy S23 Ultra cho các thuộc tính bị thiếu: ['time', 'refresh_rate']
Scraping Samsung Galaxy S23 cho các thuộc tính bị thiếu: ['time', 'operating_system', 'refresh_rate']
Scraping iPhone 8 Plus cho các thuộc tính bị thiếu: ['time', 'refresh_rate']
Scraping Samsung Galaxy S22 Ultra cho các thuộc tính bị thiếu: ['time']
Scraping iPhone XS Max cho các thuộc tính bị thiếu: ['RAM', 'time', 'refresh_rate']
Scraping iPhone 12 Pro Max cho các thuộc tính bị thiếu: ['time']
Scraping iPhone 7 Plus cho các thuộc tính bị thiếu: ['RAM', 'time', 'display_technology', 'refresh_rate']
Scraping iPhone 12 Pro cho các thuộc tính bị thiếu: ['RAM', 'time', 'display_technology', 'refresh_rate']
Scraping vivo Y18S cho các thuộc tính bị thiếu: ['time', 'refresh_rate']
Scraping iPhone 11 Pro cho các thuộc tính bị thiếu: ['refresh_rate']
Scraping iPhone X cho các thuộc tính bị thiếu: ['RAM', 'time', 'display_technology', 'refresh_rate']
Scraping iPhone 8 cho các thuộc tính bị thiếu: ['RAM', 't

Lưu dữ liệu sau điền thiếu

In [57]:
# Xuất DataFrame ra file CSV
data.to_csv('updated_data.csv', index=False, encoding='utf-8-sig')

# Xử lí cột brand

In [76]:
data = pd.read_csv('updated_data.csv')

In [59]:
print(data['brand'].unique())

['iPhone' 'OPPO' 'Samsung' 'Apple' 'vivo' 'realme' 'Xiaomi' 'Vivo' 'Điện'
 'Nubia' 'Realme' 'Nothing' 'Tecno' 'Infinix' 'TECNO' 'ASUS' 'POCO' 'Itel'
 'Oppo' 'Nokia' 'OnePlus' 'INOI' 'Meizu' 'Viettel' 'Masstel' 'Samsung.'
 'HONOR.' 'Mobell.' 'Xiaomi.' 'iPhone (Apple).' 'OPPO.' 'Nokia.'
 'Masstel.' 'realme.' 'vivo.' 'Itel.' nan]


Cần thống nhất nội dung

In [60]:
# Xử lí cột 'brand'
data['brand'] = data['name'].str.split(' ').str[0]
data.loc[data['brand'] == 'iPhone', 'brand'] = 'iPhone (Apple)'

# Xử lí cột color và image

In [61]:
print(data['color'].unique())

["['Bạc', 'Tím', 'Đen', 'Vàng']"
 "['Bạc', 'Xám', 'Xanh dương', 'Vàng', 'Xanh lá']"
 "['Hồng', 'Vàng', 'Đen']" "['Bạc', 'Đen', 'Vàng', 'Tím']"
 "['Tím', 'Đen', 'Vàng', 'Xám']"
 "['Xanh dương', 'Tím', 'Đen', 'Xám', 'Vàng', 'Xanh Lục', 'Cam']"
 "['Đen', 'Vàng', 'Tím', 'Bạc']"
 "['Xanh lá', 'Bạc', 'Xám', 'Vàng', 'Xanh dương']"
 "['Tím', 'Đen', 'Vàng', 'Bạc']"
 "['Bạc', 'Xanh lá', 'Xám', 'Vàng', 'Xanh']"
 "['Hồng', 'Đen', 'Xanh dương', 'Xanh lá', 'Vàng']"
 "['Xanh lá', 'Tím', 'Đen', 'Kem', 'Xanh', 'Đỏ', 'Xám']" "['Đen', 'Xanh']"
 "['Vàng', 'Đen']" "['Tím', 'Xám', 'Vàng', 'Đen']"
 "['Titan Xanh', 'Titan Trắng', 'Titan Tự nhiên', 'Titan Đen']"
 "['Titan Trắng', 'Titan Đen', 'Titan Xanh', 'Titan Tự Nhiên']"
 "['Đen', 'Xanh', 'Tím', 'Kem']" "['Xanh dương', 'Hồng']"
 "['Trắng', 'Xanh', 'Tím', 'Đỏ', 'Đen', 'Vàng']"
 "['Xám', 'Gold', 'Đỏ', 'Bạc']"
 "['Titan Trắng', 'Titan Tự Nhiên', 'Titan Xanh', 'Titan Đen']"
 "['Vàng', 'Bạc', 'Đen', 'Tím']"
 "['Titan Tự Nhiên', 'Titan Trắng', 'Titan Xanh', 'Tit

In [62]:
print(data['image'].unique())

["['https://cellphones.com.vn/iphone-14-pro-max-cu-tray-xuoc.html?product_id=64176', 'https://cellphones.com.vn/iphone-14-pro-max-cu-tray-xuoc.html?product_id=61437', 'https://cellphones.com.vn/iphone-14-pro-max-cu-tray-xuoc.html?product_id=60045', 'https://cellphones.com.vn/iphone-14-pro-max-cu-tray-xuoc.html?product_id=57572']"
 "['https://cellphones.com.vn/iphone-13-pro-max-128gb-cu-tray-xuoc.html?product_id=55443', 'https://cellphones.com.vn/iphone-13-pro-max-128gb-cu-tray-xuoc.html?product_id=51283', 'https://cellphones.com.vn/iphone-13-pro-max-128gb-cu-tray-xuoc.html?product_id=48422', 'https://cellphones.com.vn/iphone-13-pro-max-128gb-cu-tray-xuoc.html?product_id=47724', 'https://cellphones.com.vn/iphone-13-pro-max-128gb-cu-tray-xuoc.html?product_id=48951']"
 "['https://cellphones.com.vn/oppo-find-n3-flip-12gb-256gb-cu-dep.html?product_id=83124', 'https://cellphones.com.vn/oppo-find-n3-flip-12gb-256gb-cu-dep.html?product_id=76930', 'https://cellphones.com.vn/oppo-find-n3-flip-12

Với dữ liệu lấy từ cellphones thì color và image là một list gốm nhiều màu và ảnh khác nhau -> Xử lí cột 'color' và 'image' bằng cách: với mỗi màu trong 1 mẫu điện thoại tạo một dòng dữ liệu mới kèm với hình ảnh tương ứng.  
Với dữ liệu lấy từ thế giới di động, loại bỏ chuỗi "Màu: "

In [63]:
# Loại bỏ chuỗi "Màu: " trong cột 'color'
data['color'] = data['color'].str.replace('Màu: ', '', regex=False)

In [64]:
# Xử lý cột 'color' và 'image': với mỗi màu trong 1 mẫu điện thoại tạo một dòng dữ liệu mới kèm với hình ảnh tương ứng

# Khởi tạo các cột mới để lưu các list đã parse
data['parsed_color_list'] = [[] for _ in range(len(data))]
data['parsed_image_list'] = [[] for _ in range(len(data))]


for index, row in data.iterrows():
    # Xử lý cột 'color'
    color_str = str(row['color']).strip()
    
    if color_str.lower() in ['', 'nan', 'none']:
        parsed_colors = []
    elif color_str.startswith('[') and color_str.endswith(']'):
        parsed_list = ast.literal_eval(color_str)
        if isinstance(parsed_list, list):
            cleaned_colors = [str(c).strip() for c in parsed_list if str(c).strip() != '']
            parsed_colors = cleaned_colors
        else:
            parsed_colors = []
    else:
        parsed_colors = [c.strip() for c in color_str.split(',') if c.strip() != '']
    data.at[index, 'parsed_color_list'] = parsed_colors

    # Xử lý cột 'image'
    img_url_str = str(row['image']).strip()

    if img_url_str.lower() in ['', 'nan', 'none']:
        parsed_images = []
    elif img_url_str.startswith('[') and img_url_str.endswith(']'):
        parsed_list = ast.literal_eval(img_url_str)
        if isinstance(parsed_list, list):
            cleaned_urls = [str(u).strip() for u in parsed_list if str(u).strip() != '']
            parsed_images = cleaned_urls
        else:
            parsed_images = []
    else:
        urls = [u.strip() for u in img_url_str.split(',') if u.strip() != '']
        parsed_images = urls
    data.at[index, 'parsed_image_list'] = parsed_images

# Tạo một cột tạm thời chứa các list of tuples
data['color_image_list_of_tuples'] = [[] for _ in range(len(data))]

for index, row in data.iterrows():
    colors = row['parsed_color_list']
    images = row['parsed_image_list']
    
    len_colors = len(colors)
    len_images = len(images)

    if len_colors == 0 and len_images == 0:
        data.at[index, 'color_image_list_of_tuples'] = [(np.nan, np.nan)]
    # Nếu chỉ có màu, điền NaN cho ảnh
    elif len_colors > 0 and len_images == 0:
        data.at[index, 'color_image_list_of_tuples'] = [(color, np.nan) for color in colors]
    # Nếu chỉ có ảnh, điền NaN cho màu
    elif len_images > 0 and len_colors == 0:
        data.at[index, 'color_image_list_of_tuples'] = [(np.nan, image) for image in images]
    # Độ dài không khớp:
    else:
        max_len = max(len_colors, len_images)
        temp_tuples = []
        for i in range(max_len):
            color_val = colors[i] if i < len_colors else np.nan
            image_val = images[i] if i < len_images else np.nan
            temp_tuples.append((color_val, image_val))
        data.at[index, 'color_image_list_of_tuples'] = temp_tuples

# Explode cột chứa list các tuple
data_exploded = data.explode('color_image_list_of_tuples')

# Loại bỏ các cột không cần thiết
data_exploded = data_exploded.drop(columns=[
    'image',
    'color',
    'parsed_color_list',
    'parsed_image_list',
], errors='ignore')

# Tạo các cột 'color' và 'image' mới từ cột tuple đã explode
data_exploded['color'] = data_exploded['color_image_list_of_tuples'].apply(lambda x: x[0])
data_exploded['image'] = data_exploded['color_image_list_of_tuples'].apply(lambda x: x[1])
data_exploded = data_exploded.drop(columns='color_image_list_of_tuples')
data = data_exploded.copy()

# Xử lí cột condition

In [65]:
print(data['condition'].unique())

['Cũ Trầy Xước' 'Cũ trầy xước' 'Cũ Đẹp' 'Cũ Xước Cấn' 'Cũ đẹp'
 'Hàng trưng bày' nan '256GB)' 'Đã Kích Hoạt' 'Đã kích hoạt'
 'Chính hãng (VN/A) Đã kích hoạt' 'Cũ xước cấn' 'đã kích hoạt'
 'Cũ Trầy xước' 'Đã kích hoạt bảo hành VN/A' '128GB)' '512GB)'
 'Có trầy xước']


Cần thống nhất nội dung trong thuộc tính

In [66]:
condition_normalization_map = {
    'cũ trầy xước': 'Cũ trầy xước',
    'cũ xước cấn': 'Cũ trầy xước cấn',
    'cũ đẹp': 'Cũ đẹp',
    'hàng trưng bày': 'Hàng trưng bày',
    'đã kích hoạt': 'Đã kích hoạt',
    'chính hãng (vn/a) đã kích hoạt': 'Đã kích hoạt (Chính Hãng VN/A)',
    'đã kích hoạt bảo hành vn/a': 'Đã kích hoạt (Bảo Hành VN/A)',
    'đổi bảo hành': 'Đổi bảo hành',
    'đổi bảo hành (vn/a)': 'Đổi Bảo Hành (VN/A)',
    'đổi bảo hành vn/a': 'Đổi Bảo Hành (VN/A)',
    'xước cấn': 'Cũ trầy xước cấn',
    'cũ': 'Cũ',
    'mới fullbox': 'Mới Fullbox',
    'like new 99%': 'Like New 99%',
    'chính hãng đã kích hoạt': 'Đã kích hoạt (Chính Hãng)',
    '256gb)': 'Còn nguyên dung lượng lưu trữ',
    '512gb)': 'Còn nguyên dung lượng lưu trữ',
    '1tb)': 'Còn nguyên dung lượng lưu trữ',
    '128gb)': 'Còn nguyên dung lượng lưu trữ',
    'có trầy xước': 'Cũ trầy xước'
}
condition = data['condition']
data['condition'] = data['condition'].astype(str).str.lower()
data['condition'] = data['condition'].map(condition_normalization_map) 
data['condition'] = data['condition'].fillna(condition)
data['condition'] = data['condition'].str.strip()

# Xử lí cột CPU

In [67]:
print(data['CPU'].unique())

['Apple A16 Bionic 6-core' 'Apple A15' 'MediaTek Dimensity 9200'
 'Exynos 2400' 'Snapdragon 8 Gen 3 For Galaxy' 'Apple A16 Bionic 6 nhân'
 'Apple A16 Bionic' 'Snapdragon 8 Gen 2 (4 nm)'
 'Qualcomm Snapdragon 680, tối đa 2.4GHz' 'Snapdragon 8 Gen 2 8 nhân'
 'A17 Pro' 'Snapdragon 8 Gen 2 for Galaxy' 'Apple A18' 'A13 Bionic'
 'Apple A11 Bionic APL1W72' 'Qualcomm Snapdragon 8 Gen 1 (4 nm)'
 'Apple A12 Bionic 6 nhân' 'Apple A14 Bionic (5 nm)'
 'Snapdragon 8 Gen 2 for Galaxy (4nm) 8 nhân' 'Apple A10 Fusion APL1W24'
 'Qualcomm Snapdragon 7 Gen 1 (4 nm)' 'Apple A15 Bionic' 'Apple A18 Pro'
 'Exynos 1380' 'Helio G85' 'UNISOC T612' 'MediaTek Dimensity 7025 - Ultra'
 'Mediatek MT8781 Helio G99 (6nm)' 'Exynos 1580' 'Snapdragon 8 Gen 2'
 'Apple A13 Bionic (7 nm+)' 'Chip Snapdragon® 8 thế hệ thứ 2'
 'Exynos 1480 4nm 2.4GHz' 'Snapdragon 8 Plus Gen 1'
 'MediaTek Dimensity 9400' 'Dimensity 8300-Ultra' 'Exynos 2100 (5nm)'
 'Snapdragon 778G 5G 8 nhân' 'Snapdragon 8 Gen 3' 'Exynos 2400e (4nm)'
 'Exynos 220

Cần loại bỏ các thông tin không cần thiết, sửa lỗi chính tả.

In [68]:
cpu = data['CPU']
data['CPU'] = data['CPU'].fillna('')
data['CPU'] = data['CPU'].str.split(',').str[0].str.strip()
data['CPU'] = data['CPU'].str.replace(r'\s*\(.*?\)\s*', ' ', regex=True)
data['CPU'] = data['CPU'].str.replace(r'[^\w\s\+\.]', ' ', regex=True)
data['CPU'] = data['CPU'].str.replace(r'\s+', ' ', regex=True).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bQualcomm\s+SM\d+\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\b(thế hệ|gen|GEN)\b', 'Gen', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bcore\b', 'nhân', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bQualcomm\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bthứ\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bMobile\s+Platform\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\b(dành cho|For)\b', 'for', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'^\s*(?:Chip|Mật độ)\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\s*(?:\d+\s*nm\+?|tăng)\s*.*$', '', regex=True, flags=re.IGNORECASE).str.strip()
data['CPU'] = data['CPU'].str.replace(r'\bnapdragon\b', 'Snapdragon', regex=True, flags=re.IGNORECASE).str.strip()

cleaned_cpus = []
for cpu_entry in data['CPU']:
    if pd.isna(cpu_entry) or cpu_entry == '':
        cleaned_cpus.append(np.nan)
        continue

    final_cpu_name = re.sub(r'(Gen)(\d+)', r'\1 \2', str(cpu_entry), flags=re.IGNORECASE)
    cleaned_cpus.append(final_cpu_name.strip())

data['CPU'] = cleaned_cpus
data['CPU'] = data['CPU'].fillna(cpu)

# Xử lí cột warranty

In [69]:
print(data['warranty'].unique())

['6 tháng' nan '12 tháng' '1 tháng tại TGDĐ' 'Chính hãng đến 14/09/2025'
 'Chính hãng đến 28/10/2025' 'Chính hãng đến 14/12/2025'
 'Chính hãng đến 13/12/2025' 'Chính hãng đến 08/12/2025'
 'Chính hãng đến 30/12/2025' 'Chính hãng đến 31/12/2025'
 'Chính hãng đến 13/01/2026' 'Chính hãng đến 05/01/2026'
 'Chính hãng đến 26/01/2026' 'Chính hãng đến 24/01/2026'
 'Chính hãng đến 25/01/2026' 'Chính hãng đến 10/02/2026'
 'Chính hãng đến 28/02/2026' 'Chính hãng đến 01/03/2026'
 'Chính hãng đến 16/03/2026' 'Chính hãng đến 12/04/2026'
 'Chính hãng đến 29/04/2026' 'Chính hãng đến 22/02/2026'
 'Chính hãng đến 17/07/2026' 'Chính hãng đến 27/02/2026'
 'Chính hãng đến 09/08/2025' 'Chính hãng đến 16/08/2025'
 'Chính hãng đến 23/08/2025' 'Chính hãng đến 23/07/2025'
 'Chính hãng đến 15/08/2025' 'Chính hãng đến 17/08/2025'
 'Chính hãng đến 04/08/2025' 'Chính hãng đến 30/07/2025'
 'Chính hãng đến 31/08/2025' 'Chính hãng đến 20/08/2025'
 'Chính hãng đến 03/08/2025' 'Chính hãng đến 22/07/2025'
 'Chính hãng đế

Cần loại bỏ các chuỗi không cần thiết trong cột warranty

In [70]:
data['warranty'] = data['warranty'].str.replace(' tại TGDĐ', '', regex=False)
data['warranty'] = data['warranty'].str.replace('Chính hãng đến ', '', regex=False)

# Xử lí cột battery

In [71]:
print(data['battery'].unique())

['4.352 mAh' '4,325mAh' 'Li-Po 4300 mAh' '4,900mAh' '5,000mAh'
 'Công nghệ sạc Sạc nhanh 20W Sạc không dây MagSafe 15WSạc không dây Qi 7,5WCổng sạc Lightning'
 'Công nghệ sạc Sạc nhanh Sạc MagSafe lên đến 15WCổng sạc Lightning'
 'Công nghệ sạc Sạc nhanhSạc không dây MagSafe lên đến 15WSạc không dây Qi lên đến 7,5WCổng sạc USB Type-C'
 '5.000mAh' '5000 mAh' '4805 mAh'
 'Công nghệ sạc Sạc nhanhSạc không dây 15WSạc không dây Qi 7.5WCổng sạc USB Type-C'
 '3900 mAh'
 'Công nghệ sạc Sạc không dây lên đến 25W Sạc không dây Qi2 lên đến 15WSạc không dây Qi lên đến 7,5WCổng sạc USB Type-C'
 '3110 mAh' 'Li-ion 2691 mAh' 'Li-Ion 5000 mAh' 'Li-ion'
 'Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB Power Delivery 2.0'
 '4400 mAh' 'Li-ion 2900 mAh' '3,279mAh'
 'Công nghệ sạc Sạc không dây MagSafe lên đến 25W với bộ tiếp hợp 30W trở lênSạc không dây Qi2 lên đến 15WSạc không dây Qi lên đến 7,5WCổng sạc USB Type-C'
 'Cổng sạc USB Type-C'
 'Công nghệ sạc Sạc nhanh 20WSạc không dây chuẩn Magsafe 15wSạc khôn

Dữ liệu trong cột battery không thống nhất theo đơn vị mAh và loại bỏ thông tin không cần thiết

## Với dữ liệu từ thế giới di động

In [72]:
# Thống nhất dữ liệu trong cột 'battery' sang mAh
condition = data['battery'] == '27 giờ'
data.loc[condition, 'battery'] = '4674 mAh'
condition = data['battery'] == '26 giờ'
data.loc[condition, 'battery'] = '4005 mAh'
condition = data['battery'] == '22 giờ'
data.loc[condition, 'battery'] = '3561 mAh'
condition = data['battery'] == '33 giờ'
data.loc[condition, 'battery'] = '4685 mAh'
condition = data['name'] == 'Điện thoại iPhone 16 Pro 512GB'
data.loc[condition, 'battery'] = '3582 mAh'

# Chuyển đổi cột 'battery' từ object sang kiểu số
data['battery'] = data['battery'].str.replace(' mAh', '', regex=False)
data['battery'] = pd.to_numeric(data['battery'], errors='coerce').astype('float64')

## Với dữ liệu từ cellphones

In [73]:
# Thống nhất dữ liệu trong cột 'battery' sang mAh
data['battery'] = data['battery'].fillna('').astype(str).str.lower()
data['battery'] = data['battery'].str.replace(r'\s+', '', regex=True)
data['battery'] = data['battery'].str.replace(r'[,.]', '', regex=True)
extracted_mah = data['battery'].str.extract(r'(\d+)mah', flags=re.IGNORECASE, expand=False)
data['battery'] = pd.to_numeric(extracted_mah, errors='coerce')

# Lọc các sản phẩm có 'battery' là NaN sau chuyển đổi
products_with_nan_battery = data[data['battery'].isna()]
unique_names_with_nan_battery = products_with_nan_battery['name'].unique()
# In ra name của các sản phẩm đó
if unique_names_with_nan_battery.size > 0:
    print("\nSản phẩm có giá trị pin (mAh) là NaN:")
    for name in unique_names_with_nan_battery:
        print(f"- {name}")
else:
    print("\nKhông có sản phẩm nào có giá trị pin (mAh) là NaN.")

# Điền các trị NaN
lookup_data = {
    'product_name_lookup': ['iPhone 14 Pro Max', 'iPhone 14 Pro', 'iPhone 15 Plus', 'iPhone 15 Pro',
                            'iPhone 15 Pro Max', 'iPhone 16 Plus', 'iPhone XS Max', 'iPhone 12 Pro Max',
                            'iPhone 16 Pro Max', 'iPhone 15', 'iPhone 14 Plus', 'iPhone 12 Pro', 
                            'iPhone 12', 'iPhone 16 Pro', 'iPhone 16', 'iPhone XS', 'iPhone 12 mini',
                            'iPhone 13', 'realme 9i', 'iPhone 14', 'OPPO Reno 13 5G',
                            'Xiaomi Redmi Note 14', 'Sony Xperia 10V', 'OPPO Reno13 Pro',
                            'Xiaomi Poco X7 Pro 5G', 'Xiaomi Redmi 10C 4G', 'Xiaomi Redmi Note 14 Pro',
                            'Xiaomi Mi 11T'],
    'correct_battery': [4323, 3200, 4383, 3274, 4422, 4674, 3174, 3687, 4685, 3349, 4325, 2815, 2815,
                       4674, 3561, 2658, 2227, 3240, 5000, 3279, 5600, 5500, 5000, 5800, 6000, 5000,
                       5500, 5000]
}
lookup = pd.DataFrame(lookup_data)

battery_map = lookup.set_index('product_name_lookup')['correct_battery']
data['battery'] = data['battery'].fillna(data['name'].map(battery_map))


Sản phẩm có giá trị pin (mAh) là NaN:
- iPhone 14 Pro Max
- iPhone 13 Pro Max
- OPPO Find N3 Flip
- Samsung Galaxy S24 Plus
- Samsung Galaxy S24 Ultra
- iPhone 14 Pro
- iPhone 15 Plus
- Samsung Galaxy S23 Ultra
- OPPO A78
- OPPO Find N3
- iPhone 15 Pro
- iPhone 15 Pro Max
- Samsung Galaxy S23
- iPhone 16 Plus
- iPhone 11
- iPhone 8 Plus
- Samsung Galaxy S22 Ultra
- iPhone XS Max
- iPhone 12 Pro Max
- Samsung Galaxy Z Fold5
- iPhone 7 Plus
- Samsung Galaxy M55
- iPhone 14
- iPhone 16 Pro Max
- iPhone 15
- iPhone 14 Plus
- iPhone 13
- Samsung Galaxy A35 5G
- iPhone 12 Pro
- vivo Y18S
- iPhone 16 Pro
- realme Note 60
- Xiaomi Redmi Note 14 5G
- iPhone 13 mini
- iPhone 16
- iPhone 11 Pro
- iPhone X
- iPhone 8
- iPhone 13 Pro
- vivo V27e
- iPhone 12
- iPhone XS
- iPhone 12 mini
- Samsung Galaxy A56 5G
- iPhone SE
- Xiaomi 13
- Samsung Galaxy A55 5G
- Samsung Galaxy Z Fold4
- OPPO Find X8
- Xiaomi Poco X6 Pro 5G
- Samsung Galaxy S21 FE
- iPhone 13 VN/A
- Samsung Galaxy A73 5G
- Samsung Gala

# Xử lí cột RAM và capacity

In [74]:
print(data['RAM'].unique())

['6 GB' '12 GB' '8 GB' '16 GB' '4 GB' '3 GB' '4GB (LPDDR4)' '3GB (LPDDR4)'
 '6GB (LPDDR4)' '2GB (LPDDR4)' '3GB' '8GB + Mở rộng 8GB' '2 GB'
 '4GB + Mở rộng 4GB' nan '48MB' '48 MB' '64 MB' '16 MB' '0.4GB']


Dữ liệu trong cột RAM không thống nhất theo đơn vị GB.

In [75]:
print(data['capacity'].unique())

['128 GB' '256 GB' '512 GB' '64 GB' '32 GB' '1 TB' '16 GB' nan '128 MB'
 '16 MB']


Dữ liệu theo cột capacity không thống nhất theo đơn vị GB.

In [192]:
# Hàm chuyển đơn vị sang GB và chuyển đổi cột sang float
def convert_to_gb(value):
    if pd.isna(value) or str(value).lower().strip() == 'nan':
        return np.nan
    value = str(value).upper().strip()
    if 'GB' in value:
        return float(value.replace('GB', '').strip())
    elif 'MB' in value:
        return float(value.replace('MB', '').strip()) / 1024
    elif 'TB' in value:
        return float(value.replace('TB', '').strip()) * 1024
    else:
        return np.nan

# Xử lý các trường hợp có RAM mở rộng được
data['RAM'] = data['RAM'].copy()

plus_mask = data['RAM'].str.contains(r'\+', regex=True) | data['RAM'].str.contains(r'mở rộng', regex=True)
if plus_mask.any():
    data.loc[plus_mask, 'RAM'] = data.loc[plus_mask, 'RAM'].apply(
        lambda x: f"{sum(int(val) for val in re.findall(r'(\d+)GB', x))} GB"
    )

# Chuyển đổi cột 'capacity' từ object sang float
if data['capacity'].dtype == object:
    data['capacity_GB'] = data['capacity'].apply(convert_to_gb)
    data['capacity'] = data['capacity_GB']
    data.drop(columns='capacity_GB', inplace=True)

# Chuyển đổi cột 'RAM' từ object sang float
if data['RAM'].dtype == object:
    data['RAM_GB'] = data['RAM'].apply(convert_to_gb)
    data['RAM'] = data['RAM_GB']
    data.drop(columns='RAM_GB', inplace=True)

# Xử lí cột screen size

In [193]:
print(data['screen_size'].unique())

['6.7 inches' '6.8 inches' '6.1 inches' '6.4 inches' '7.82 inches'
 '5.5 inches' '6.5 inches' '7.6 inches' '6.9 inches' '6.6 inches'
 '6.73 inches' '6.56 inches' '6.3 inches' '6.74 inches' '6.67 inches'
 '5.4 inches' '5.8 inches' '4.7 inches' '6.62 inches' '6.36 inches'
 '6.59 inches' '6.72 inches' '6.88 inches' '6.66 inches' '6.43 inches'
 '6.79 inches' '6.71 inches' '6.78 inches' '3 inches' '6.64 inches'
 '1.77 inches' '6.75 inches' '6.55 inches' '6.745 inches' '2.4 inches'
 '6.2 inches' '10.2 inches' '6.52 inches' '10.1 inches' '1.8 inches'
 '6.44 inches' '6.53 inches' '6.58 inches' '6.51 inches' '6.95 inches'
 '7,92 inches' nan '6.28 inches' '5.7 inches' '2.0 inches' '6.81 inches'
 '8.12 inches' '2.8 inches' 'Chính 6.7" & Phụ 3.4"' '6.56"' '1.77"' '6.2"'
 '6.4"' '6.7"' '6.8"' '6.6"' '6.5"' 'Chính 7.6" & Phụ 6.3"' '6.73"' '6.1"'
 '1.8"' '6.36"' '2"' '6.67"' '6.79"' '6.71"' '6.88 "' '6.43"' '6.72"'
 '2.4"' '6.74"' '2.31"' '6.68 "' '6.9"' '6.64"' '2.8"' '6.745 "'
 'Chính 6.8" & Phụ 3.

Cần loại bỏ inches và ", chỉ lấy kích thước màn hình chính

In [194]:
data['screen_size'] = data['screen_size'].apply(
    lambda x: x.split('&')[0].split('Chính')[1].strip() 
    if isinstance(x, str) and 'Chính' in x and len(x.split('Chính')) > 1
    else x if isinstance(x, str) else None
)

# Chuyển cột 'screen_size' sang kiểu dữ liệu số
data['screen_size'] = data['screen_size'].astype(str)
data['screen_size'] = data['screen_size'].str.replace(r'"', '', regex=True).str.strip()
data['screen_size'] = data['screen_size'].str.replace(r'\s*inches|\s*inch', '', regex=True)
data['screen_size'] = pd.to_numeric(data['screen_size'], errors='coerce')

# Xử lý cột screen resolution

In [195]:
print(data['screen_resolution'].unique())

[nan '2778 x 1284 pixel' '1080 x 2520 pixels'
 '3120 x 1440 pixels (Quad HD+)' '1440 x 3120 pixels' '2796 x 1290-pixel'
 '2556 x 1179 pixels' '1440 x 3088 pixels (QHD+)'
 '1080 x 2400 pixels (FullHD+)' '2268 x 2440 pixels' '1792 x 828 pixel'
 '1080 x 1920 pixels (FullHD)' '1242 x 2688 pixel' '1284 x 2778 pixels'
 '2176 x 1812 pixels (QXGA+)' '2868 x 1320 pixels' '2532 x 1170 pixels'
 '1080 x 2340 pixels (FullHD+)' '1220 x 2712 pixels' '1080 x 2640 pixels'
 '1170 x 2532 pixels' '1612 × 720 pixel' '3200 x 1440 pixel'
 '2622 x 1206  pixels' '1600 x 720 pixels (HD+)' '2436 x 1125 pixels'
 '1125 x 2436 pixels' '750 x 1334 pixels' '1334 x 750 pixels (HD+)'
 '1256 x 2760 pixels' '1612x720 (HD+)' '2688 x 1242 pixels'
 '2160 x 1856 (QXGA+)' '2712 x 1220 pixels' '720 x 1640 pixels'
 '720 x 1612 (HD+)' '1080 x 2412 pixels' '3840 x 1644 pixels'
 '1080 x 2408 pixels' '2460x1080 pixels' '720 x 1600 pixel'
 '720 x 1560 pixes' '1240 x 2772 pixels' '1650x720 pixels'
 '1080 x 2448 pixels (FullHD+)' '108

Cần loại bỏ thông tin không cần thiết, tách cột

In [196]:
# Xử lí thông tin độ phân giải trong cột 'screen_resolution'
data['screen_resolution_temp'] = data['screen_resolution'].apply(
    lambda x: x.split('x P')[0].split('Chính: ')[1].strip()
    if isinstance(x, str) and 'Chính: ' in x
    else x if isinstance(x, str) else None
)

# Tách cột 'screen_resolution' thành 'resolution_width' và 'resolution_height'
data[['resolution_width', 'resolution_height']] = data['screen_resolution_temp'].str.extract(r'(?:.*\(|)(\d+) x (\d+)')

# Chuyển đổi cột 'resolution_width' sang kiểu số
data['resolution_width'] = data['resolution_width'].astype(str)
data['resolution_width'] = pd.to_numeric(data['resolution_width'], errors='coerce')

# Chuyển đổi cột 'resolution_height' sang kiểu số
data['resolution_height'] = data['resolution_height'].astype(str)
data['resolution_height'] = pd.to_numeric(data['resolution_height'], errors='coerce')

# Xóa các cột không cần thiết.
data = data.drop(columns=['screen_resolution_temp'])

# Xử lý cột operating system

In [197]:
print(data['operating_system'].unique())

['iOS 16' 'iOS15' 'Android 13' 'Android 14, One UI 6.1' 'iOS 17'
 'Android 13, One UI 5' 'ColorOS 13.1.1, nền tảng Android 13' nan 'iOS 18'
 'iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành)' '11'
 'Android 12, One UI 4.1' '12'
 'iOS 14.1 hoặc cao hơn (Tùy vào phiên bản phát hành)' 'Android 14'
 'iOS 15' 'Xiaomi HyperOS' 'iOS' 'Funtouch OS 13' 'Android' 'iOS 13'
 'MIUI 14, Android 13' 'Android 10, One UI 2.1' 'Android 12, One UI 4.1.1'
 'ColorOS 15, nền tảng Android 15' 'Android 17' 'Android 11, MIUI 12.5'
 'Xiaomi HyperOS 2' 'iOS 12' 'Android 11' 'Android™ 13'
 'Android 12, ColorOS 12.1' 'Android 12' 'Android 10'
 'MIUI 13 dựa trên Android 11' 'Xiaomi HyperOS (Android 14)'
 'Android 14, One UI 6' 'Android 14 OS' 'S30+'
 'realme UI 4.0 Trên nền tảng Android 13' 'Android 14, Funtouch OS 14'
 'Android U' 'Android 11, MIUI 12' 'Android 13.0'
 'Android 11, ColorOS 11.1' 'Android T' 'Android 13, realme UI'
 'ColorOS 12.1, nền tảng Android 12' 'Android 11, One UI 3.0' 'Android Go'
 'Android

In [198]:
data.loc[data['operating_system'] == '11', 'operating_system'] = 'iOS 11'
data.loc[data['operating_system'] == '12', 'operating_system'] = 'iOS 12'

# Xử lý cột SIM

In [199]:
print(data['SIM'].unique())

['2 SIM (nano‑SIM và eSIM)' '2 SIM (Nano-SIM)'
 'SIM 1 + SIM 2 / SIM 1 + eSIM / 2 eSIM' '2 Nano-SIM + eSIM'
 'Nano-SIM + eSIM' nan 'Sim kép (nano-Sim và e-Sim) - Hỗ trợ 2 e-Sim'
 'Nano-SIM' '2 Nano-SIM' 'Dual nano-SIM hoặc 1 nano-SIM + 1 eSIM'
 '2 Nano SIM hoặc 1 Nano + 1 eSIM' 'Micro-SIM'
 '2 Nano SIM (Sim 2 chung khe với thẻ nhớ)' 'Micro-SIM + Nano SIM'
 '1 Nano SIM & 1 eSIM' '1 Nano SIM + 1 eSIM hoặc 2 eSIM' '2 Nano SIM'
 '2 Nano SIM hoặc 2 eSIM hoặc 1 Nano SIM + 1 eSIM'
 '2 Nano SIM hoặc 1 Nano SIM + 1 eSIM'
 '2 Nano SIM (SIM 2 chung khe thẻ nhớ)' '2 Nano SIM + 1 eSIM'
 '2 Micro SIM']


In [200]:
# Xử lí thông tin cột 'SIM'
data['processed_SIM'] = data['SIM'].astype(str).str.lower()
data['processed_SIM'] = data['processed_SIM'].replace('nan', np.nan)

# Tạo cột 'has_nano_sim' (nếu có là 1 - không có là 0)
data['has_nano_sim'] = data['processed_SIM'].str.contains(r'nano[-]?sim', case=False, na=np.nan).astype(float)

# Tạo cột 'has_esim' (nếu có là 1 - không có là 0)
data['has_esim'] = data['processed_SIM'].str.contains(r'e[-]?sim', case=False, na=np.nan).astype(float)

# Cập nhật regex để bắt 'SIM' hoặc 'SIM 1', 'SIM 2',
mask_generic_sim = (
    data['processed_SIM'].notna() &
    data['processed_SIM'].str.contains(r'\bSIM(?: \d+| vật lý)?\b', case=False, na=False)
)

data = data.drop(columns = 'processed_SIM')

# Cập nhật has_nano_sim thành 1.0 cho các trường hợp này
data.loc[mask_generic_sim, 'has_nano_sim'] = 1.0

# Xử lý cột display technology

In [201]:
print(data['display_technology'].unique())

['Super Retina XDR OLED' 'OLED' 'AMOLED' 'Dynamic AMOLED'
 'Dynamic AMOLED 2X' 'IPS LCD' nan 'Super Retina OLED' 'Super AMOLED Plus'
 'Super AMOLED' 'Super Retina XDR'
 'Cảm ứng điện dung LED-backlit IPS LCD, 16 triệu màu'
 'Retina IPS LCD capacitive touchscreen, 16M colors, Wide color gamut, True-tone, 625 nits max brightness'
 'Dynamic AMOLED 2X, 16M colors, Corning Gorilla Glass Victus, HDR10+, Always-on display, 120Hz@FHD/60Hz@QHD refresh rate'
 'Liquid Retina' 'Liquid Retina LCD' 'Flexible AMOLED On-cell' 'TFT LCD'
 'LCD' 'PLS LCD' 'LTPO OLED' 'Fluid AMOLED' 'sAMOLED FHD+'
 'CrystalRes AMOLED' 'LED-backlit IPS LCD, HD' 'LTPO AMOLED'
 'Super Retina XDR OLED, HDR10, Dolby Vision, Wide color gamut, True-tone'
 'Liquid Retina IPS LCD capacitive touchscreen, 16M colors'
 'Dynamic AMOLED QHD+ Infinity-O'
 'Dynamic AMOLED 2X, 120Hz(1080p), HDR10+, Gorilla Glass 6'
 'Super AMOLED Plus, Corning Gorilla Glass 5, HDR10+, Always-on display'
 'LTPS LCD' 'TN']


Cần loại bỏ các thông tin không cần thiết 

In [202]:
display_info = data['display_technology']
data['display_technology'] = data['display_technology'].fillna('').astype(str)
data['display_technology'] = data['display_technology'].str.extract(r'(.*?lcd)', flags=re.IGNORECASE, expand=False)
data['display_technology'] = data['display_technology'].fillna(display_info)
data['display_technology'] = data['display_technology'].str.split(',').str[0]
data['display_technology'] = data['display_technology'].str.replace('Cảm ứng điện dung ', '', regex=False)
data['display_technology'] = data['display_technology'].str.strip()

# Xử lý cột size

In [203]:
print(data['size'].unique())

['160.7 x 77.6 x 7.9 mm' '160.8 x 78.1 x 7.65mm'
 'Khi mở: 166.42 x 75.78 x 7.79 mm Khi gập: 85.54 x 75.78 x 16.45 mm'
 '158.5 x 75.9 x 7.75mm' '162.3 x 79.0 x 8.6mm'
 'Dài 174.5mm - Ngang: 71.5mm - Dày 7.85mm' '160,9 x 77,8 x 7,80 mm'
 '163.4 x 78.1 x 8.9 mm'
 'Đen: 160 x 73.23 x 7.93 mm  Xanh: 160 x 73.23 x 7.99 mm'
 'Màu Vàng Khi mở: 153.4 x 143.1 x 5.9 (màn hình ngoài)/5.8 (mặt lưng) mm Khi gập: 153.4 x 73.3 x 11.7 mm  Màu Đen Khi mở: 153.4 x 143.1 x 5.9 (màn hình ngoài)/6.0 (mặt lưng) mm Khi gập: 153.4 x 73.3 x 11.9 mm"'
 '146,6 x 70,6 x 8,25 mm' '159,9 x 76,7 x 8,25 mm' nan
 '150.9mm - 75.7mm - 8.3mm'
 '158.4 x 78.1 x 7.5 mm (6.24 x 3.07 x 0.30 in)' '163.3 x 77.9 x 8.9 mm'
 '157.5 - 77.4 - 7.7 mm' '160.8 x 78.1 x 7.4 mm'
 'Mở: 154.9 x 129.9 x 6.1mm Gập: 154.9 x 67.1 x 13.4mm'
 '158.2 x 77.9 x 7.3 mm (6.23 x 3.07 x 0.29 in)' '163.9 x 76.5 x 7.8 mm'
 '163 x 77,6 x 8,25 mm' '147.6 x 71.6 x 7.80 mm'
 'Dọc 160.8 x Ngang 78.1 x Dày 7.8mm' '146,7 x 71,5 x 7,65mm'
 '161.7 x 78.0 x 8.2 mm

Chỉ lấy kích thước, tạo ra các cột mới lưu thông tin dài, rộng, cao

In [204]:
# Xử lý dữ liệu cột 'size' để tạo 3 cột mới
data['size_str_processed'] = (data['size'].astype(str).str.strip()
                              .replace(['', 'nan', 'none'], 'INVALID', regex=True))

processed_size_lists = []
for size_item_str in data['size_str_processed']:
    current_list = [np.nan] * 3
    if size_item_str != 'INVALID':
        try:
            parsed_list = ast.literal_eval(size_item_str)
            if isinstance(parsed_list, list):
                numeric_elements = [pd.to_numeric(val, errors='coerce') for val in parsed_list]
                current_list = numeric_elements[:3] + [np.nan] * (3 - len(numeric_elements))
        except (ValueError, SyntaxError):
            pass
    processed_size_lists.append(current_list)

data[['height', 'width', 'depth']] = pd.DataFrame(processed_size_lists, index=data.index)
data.drop(columns=['size_str_processed'], inplace=True)

# Chuẩn hóa cột 'size'
data['general_size_info_cleaned'] = (data['size'].astype(str)
                                     .str.replace(',', '.', regex=False)
                                     .str.replace(r'\bxx\b', 'x', regex=True)
                                     .str.replace('≈', '', regex=False)
                                     .str.strip()
                                     .replace(['nan', 'none', ''], np.nan))

# Xử lý kích thước từ mô tả text
pattern_height = re.compile(r'(?:dọc|dài|chiều dài|cao)\s*[:\-]?\s*([\d\.]+)', re.IGNORECASE)
pattern_width = re.compile(r'(?:rộng|ngang|chiều rộng)\s*[:\-]?\s*([\d\.]+)', re.IGNORECASE)
pattern_depth = re.compile(r'(?:dày|độ dày|siêu mỏng)\s*[:\-]?\s*([\d\.]+)', re.IGNORECASE)
pattern_3_dims = re.compile(r'([\d\.]+)\s*(?:mm)?\s*[-xX×*]\s*([\d\.]+)\s*(?:mm)?\s*[-xX×*]\s*([\d\.]+)', re.IGNORECASE)

dims = data['general_size_info_cleaned'].str.extract(pattern_3_dims)
data['height'] = pd.to_numeric(dims[0], errors='coerce')
data['width'] = pd.to_numeric(dims[1], errors='coerce')
data['depth'] = pd.to_numeric(dims[2], errors='coerce')

data['height'] = data['height'].fillna(data['general_size_info_cleaned'].str.extract(pattern_height)[0].astype(float))
data['width'] = data['width'].fillna(data['general_size_info_cleaned'].str.extract(pattern_width)[0].astype(float))
data['depth'] = data['depth'].fillna(data['general_size_info_cleaned'].str.extract(pattern_depth)[0].astype(float))

data.drop(columns=['general_size_info_cleaned'], errors='ignore', inplace=True)

# Xử lý cột GPU

In [205]:
print(data['GPU'].unique())

['Apple GPU (5-core graphics)' 'GPU 5 nhân' 'Immortalis-G715 MC11'
 'Adreno 750' 'Adreno 750 (1 GHz)' 'Apple GPU 5 nhân' 'GPU 5 lõi'
 'Adreno 740' 'Adreno 610' 'GPU 6 nhân mới' nan 'GPU 5 lõi mới'
 'Apple GPU (3 lõi đồ họa)' 'Adreno 730' 'Apple GPU 4 nhân'
 'Apple GPU (4-core graphics)' 'PowerVR Series7XT Plus (6 lõi đồ họa)'
 'Adreno 644' 'GPU 6 lõi mới' '5‑core GPU' '5 nhân GPU' 'Mali-G68 MP5'
 'Mali-G52 MC2' 'Qualcomm® Adreno™ GPU' 'Mali-G57' 'IMG BXM-8-256'
 'GPU 4 nhân' 'Mali-G57 MC2' 'GPU Qualcomm® Adreno™' 'Mali-G77 MP11'
 'Immortalis-G720 MC12' 'AMD Titan 1WGP' 'Immortalis G925 MC12'
 'Mali G615' 'Adreno 650' 'Xclipse 920' 'Mali-G710 MC10' 'Mali-G52'
 'Mali-G57 MP1' 'ARM Mali-G57 MC2' 'Adreno GPU' 'ARM Mail-G68 MC4' '8 lõi'
 'Mali-G68' 'Adreno 660' 'ARM Mali G57 1Core @650MHz' 'Adreno 618'
 'Adreno ™ 730' 'Mali-G57 3D Graphics' 'Mali-G615 MC2' 'Adreno 710'
 'Mali-G57 MC2, 650MHz' 'ARM Mali-G57 MP1' 'ARM G57 MC2' 'Mali-G68 MC4'
 'Adreno 642' 'ARM MAli-G57' 'ARM Mali-G57' 'Xclips

Loại bỏ các thông tin không cần thiết, sửa lỗi chính tả.

In [206]:
gpu = data['GPU']
data['GPU'] = data['GPU'].fillna('')
data['GPU'] = data['GPU'].str.replace(r'[^\w\s\+\.\-]', ' ', regex=True)
data['GPU'] = data['GPU'].str.replace(r'\s+', ' ', regex=True).str.strip()
data['GPU'] = data['GPU'].str.replace(r'\bQualcomm\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['GPU'] = data['GPU'].str.replace(r'(\d+)\s*(?:lõi|core)s?\b', r'\1 nhân', regex=True, flags=re.IGNORECASE).str.strip()
data['GPU'] = data['GPU'].str.replace(r'\b(up to|Graphics|đồ họa|mới)\s*', '', regex=True, flags=re.IGNORECASE).str.strip()
data['GPU'] = data['GPU'].str.replace(r'(\d+)-(?:nhân|lõi|core)s?\b', r'\1 nhân', regex=True).str.strip()
data['GPU'] = data['GPU'].str.replace(r'(\b\w+)\s*(-?G\d+)', r'\1\2', regex=True).str.strip()
data['GPU'] = data['GPU'].str.replace(r'\bAdreno(?:\s*\bGPU\b)?\s*(\d+)\b(?:\s*\bGPU\b)?', r'Adreno \1', regex=True, flags=re.IGNORECASE).str.strip()

cleaned_gpus = []
for gpu_entry in data['GPU']:
    if pd.isna(gpu_entry) or gpu_entry == '':
        cleaned_gpus.append(np.nan)
        continue

    processed_gpu_entry = str(gpu_entry)
    cleaned_gpus.append(processed_gpu_entry.strip())  

data['GPU'] = cleaned_gpus
data['GPU'] = data['GPU'].fillna(gpu)

# Xử lý cột bluetooth

In [207]:
print(data['bluetooth'].unique())

['v5.3, A2DP, LE' 'v5.0' 'v5.3, A2DP, LE, aptX HD' 'Bluetooth 5.3' '5.3'
 'A2DP, LE, v5.3' 'v5.3' '5.3, A2DP, LE' nan '5.0' '5.0, A2DP, LE'
 '5.2, A2DP, LE' 'LE, A2DP, v5.0' '4.2, A2DP, LE' 'v5.2, A2DP, LE'
 'v5.0, A2DP, LE' 'Bluetooth 5.0' '5.4' 'v5.2' '5.0, A2DP, LE, aptX'
 'v5.4' '5.2, A2DP, LE, aptX HD' '5.4 BLE' 'v5.4, A2DP, LE' 'Có'
 '5.1, A2DP, LE' 'Bluetooth 6.0' 'v 5.0'
 'A-GPS, A-GLONASS, Beidou, Galileo, QZSS' 'v5.1' '5.3, A2DP, LE, aptX HD'
 'Có 5.0, A2DP, LE' '5.2, A2DP, LE, aptX HD, aptX Adaptive'
 'v5.1, A2DP, LE' '5.1, A2DP, LE, aptX HD' 'v5,2'
 'GPS/AGPS,Galileo,Glonass' '5.0, A2DP, LE, aptX HD' '5.1'
 '5.2, A2DP, LE, L2HC' '5.2' 'BT5.2' 'v4.2' 'v5.1 BLE' 'v5.2, apt-X, A2DP'
 '5.1, A2DP, aptX HD, LE' 'Bluetooth\t5.0, BLE' '2.1' 'Bluetooth v5.4'
 'A2DPLEv5.2' 'BT 5.4' 'v5.2, A2DP, LE, aptX HD, LHDC'
 'Bluetooth 6.0, Dual-Bluetooth' 'BT5.4' '5.1, A2DP, LE5.2, A2DP, LE'
 'A2DP, LE' 'A2DP, LE, v4.2' 'v5.2, A2DP, LE, aptX HD' '4.2, A2DP'
 'v5.3 BLE' 'BT 5.2' 'Bluetooth 5.2'

Loại bỏ thông tin không cần thiết

In [208]:
data['processed_bluetooth'] = data['bluetooth'].fillna('').astype(str)
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'(\d+),(\d+)', r'\1.\2', regex=True)
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\bv\s*(\d+\.\d+)', r'v\1', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'(?<!Dual[\s\-])\bBluetooth\b\s*', '', regex=True).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\baptX\b', 'apt-X', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\bBT\s*(\d+\.\d+.*)?\b', r'\1', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'(?<!v)(\b\d+\.\d+(?:\s*[a-zA-Z0-9]+)?\b)', r'v\1', regex=True, flags=re.IGNORECASE).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r'\b(Có)\b(?!$)', '', regex=True).str.strip()
data['processed_bluetooth'] = data['processed_bluetooth'].str.replace(r',\s*', ', ', regex=True)
data['bluetooth'] = data['processed_bluetooth'].replace('', np.nan)
data = data.drop(columns=['processed_bluetooth'], errors='ignore')

# Xử lý cột refresh rate

In [209]:
print(data['refresh_rate'].unique())

['120Hz' nan '90Hz' '60Hz' 'Từ 144Hz trở lên'
 'Tần số quét Chính: 120 Hz & Phụ: 60 Hz' 'Tần số quét 90 Hz'
 'Tần số quét 120 Hz' 'Tần số quét 60 Hz' 'Tần số quét 144 Hz']


Lấy thông tin tần số quét chính, loại bỏ chữ Hz

In [210]:
# Xử lí tần số quét chính & phụ
data['refresh_rate'] = data['refresh_rate'].astype(str).str.extract(
    r"Tần số quét Chính: (\d+) Hz", flags=re.IGNORECASE, expand=False
)

# Xử lý các hàng mà ở bước xử lí trên không tìm thấy
data['refresh_rate'] = data.apply(
    lambda row: re.search(r"Tần số quét (\d+) Hz", str(row['refresh_rate']), re.IGNORECASE).group(1)
                if pd.isna(row['refresh_rate']) and isinstance(row['refresh_rate'], str)
                   and re.search(r"Tần số quét (\d+) Hz", str(row['refresh_rate']), re.IGNORECASE)
                   and "Chính" not in str(row['refresh_rate'])
                else row['refresh_rate'],
    axis=1
)

# Chuyển cột 'refresh_rate' sang kiểu dữ liệu số
data['refresh_rate'] = data['refresh_rate'].astype(str)
data['refresh_rate'] = pd.to_numeric(data['refresh_rate'], errors='coerce')

# Lưu dữ liệu đã qua xử lí

In [104]:
# Xuất DataFrame ra file CSV
data.to_csv('tidy_data.csv', index=False, encoding='utf-8')

PermissionError: [Errno 13] Permission denied: 'tidy_data.csv'

# Xử lý giá trị thiếu

In [212]:
data.isnull().sum()

name                      0
brand                     0
condition               210
price_old              3282
price_new             12697
warranty               1641
CPU                     169
RAM                     719
capacity                 27
time                   2467
battery               12519
screen_size              16
operating_system       2323
display_technology     1006
screen_resolution       457
SIM                     275
size                    386
weight                 3871
bluetooth               468
refresh_rate          14370
GPU                    2555
color                   156
image                    18
resolution_width        557
resolution_height       557
has_nano_sim            275
has_esim                275
height                11813
width                 11811
depth                 11785
dtype: int64

Cột price_new, CPU, RAM, capacity có chứa giá trị thiếu, nghi ngờ bị thiếu theo dòng máy.


In [213]:
# price_new
data['price_new_missing'] = data['price_new'].isna().astype('int')
# Tính tỷ lệ missing price_new theo tên điện thoại
rate_prc_name = data.groupby('name')['price_new_missing'].mean()
print(rate_prc_name[rate_prc_name > 0])

data = data.drop(columns=['price_new_missing'])

name
ASUS ROG Phone 5S            0.4
ASUS ROG Phone 6             0.8
ASUS ROG Phone 6 Diablo      1.0
ASUS ROG Phone 7             1.0
ASUS ROG Phone 7 Ultimate    1.0
                            ... 
vivo Y29                     1.0
vivo Y35                     1.0
vivo Y36                     0.5
vivo Y38                     1.0
vivo Y53S                    1.0
Name: price_new_missing, Length: 414, dtype: float64


Rõ ràng thiếu do phụ thuộc vào dòng máy. Qua tra cứu thì các dòng máy trên đã ngừng kinh doanh sản phẩm mới -> không có giá bán mới. 
Xử lý: Do price_new là một thuộc tính quan trọng của bài toán -> drop những dòng bị missing ở thuộc tính price_new.

In [214]:
data = data.dropna(subset=['price_new'])

In [215]:
# CPU
data['CPU_missing'] = data['CPU'].isna().astype('int')
# Tính tỷ lệ missing CPU theo tên điện thoại
rate_CPU_name = data.groupby('name')['CPU_missing'].mean()
print(rate_CPU_name[rate_CPU_name > 0])

data = data.drop(columns=['CPU_missing'])

name
INOI 288S 4G           1.0
Masstel Fami 60S 4G    1.0
Nubia Music NFC        1.0
Samsung Galaxy A33     1.0
Sony Xperia 10V        1.0
Name: CPU_missing, dtype: float64


Rõ ràng thiếu do phụ thuộc vào dòng máy. Qua tra cứu thì 2 dòng máy này không public thông số kỹ thuật CPU. 
Xử lý: Không cần xử lý, không thể điền thiếu do đây là thông số của nhà sản xuất.

In [216]:
# RAM
data['RAM_missing'] = data['RAM'].isna().astype('int')
# Tính tỷ lệ missing RAM theo tên điện thoại
rate_RAM_name = data.groupby('name')['RAM_missing'].mean()
print(rate_RAM_name[rate_RAM_name > 0])

data = data.drop(columns=['RAM_missing'])

name
INOI 288S 4G           1.000000
Masstel Fami 60S 4G    1.000000
iPhone 12              0.714286
iPhone 12 Pro          1.000000
iPhone 12 Pro Max      0.750000
iPhone 12 mini         0.666667
iPhone 13 Pro          0.125000
iPhone 14 Pro          0.600000
iPhone 14 Pro Max      0.545455
iPhone 15              0.583333
iPhone 15 Plus         0.937500
iPhone 15 Pro          1.000000
iPhone 15 Pro Max      1.000000
iPhone 16              1.000000
iPhone 16 Plus         1.000000
iPhone 16 Pro          1.000000
iPhone 16 Pro Max      1.000000
iPhone 7 Plus          1.000000
iPhone 8               1.000000
iPhone 8 Plus          0.533333
iPhone X               1.000000
iPhone XR              1.000000
iPhone XS              1.000000
iPhone XS Max          1.000000
Name: RAM_missing, dtype: float64


Rõ ràng thiếu do phụ thuộc vào dòng máy. Tương tự CPU, các dòng máy này không public thông số RAM.
Xử lý: Tương tự CPU.

In [217]:
#capacity
data['capacity_missing'] = data['capacity'].isna().astype('int')
# Tính tỷ lệ missing RAM theo tên điện thoại
rate_capacity_name = data.groupby('name')['capacity_missing'].mean()
print(rate_capacity_name[rate_capacity_name > 0])

data = data.drop(columns=['capacity_missing'])

name
INOI 288S 4G    1.0
Name: capacity_missing, dtype: float64


Rõ ràng thiếu do phụ thuộc vào dòng máy. Tương tự CPU, dòng máy này không public thông số dung lượng lưu trữ.
Xử lý: Tương tự CPU. 

## Xử lý giá trị ngoại lai

In [218]:
# Kiểm tra khoảng giá toàn cục
print("Khoảng giá toàn cục")
for k, v in price_range.items():
    print(f"{k:>5}: {int(v):,}")

Khoảng giá toàn cục


NameError: name 'price_range' is not defined

In [ ]:
# Đếm số giá khác nhau trong từng dòng máy
unique_prices_per_model = data.groupby('name')['price_new'].nunique().sort_values(ascending=False)
print("\nSố lượng giá khác nhau trong mỗi dòng máy:")
print(unique_prices_per_model)

In [ ]:
# Đếm số mẫu mỗi dòng máy
sample_count_per_model = data['name'].value_counts()
print("\nSố mẫu mỗi dòng máy:")
print(sample_count_per_model)

Giá và thông số khác biệt lớn giữa các dòng máy (từ vài trăm nghìn đến gần 50 triệu).
Đa số dòng máy có giá giao động trong khoảng nhất định.
Số lượng mẫu không đều giữa các dòng, xử lý toàn cục sẽ lệch kết quả.
-> xử lý theo từng dòng máy (name).